In [12]:
import tkinter as tk
import tkinter.font as font
import webbrowser
import numpy as np
import joblib
import time
import random # Importation ajoutée pour la mesure de sécurité

class FuturisticTicTacToe:
    """
    Une version améliorée du jeu de Morpion avec une interface futuriste
    construite en utilisant Python et le module Tkinter.

    Cette version inclut un menu principal, une refonte visuelle, et un
    plateau de jeu adaptatif qui se redimensionne avec la fenêtre tout en
    restant parfaitement carré.
    """
    def board_to_vector(self, board_list):
        """Convertit la liste du plateau en un vecteur numérique pour le modèle."""
        mapping = {'X': 1, 'O': -1, '': 0}
        return [mapping[cell] for cell in board_list]
        
    def __init__(self, root):
        """
        Initialise l'application.

        Args:
            root (tk.Tk): La fenêtre racine de l'application.
        """
        self.root = root
        self.root.title("Morpion NÉON (Python)")
        self.root.configure(bg="#0d0d0d")
        self.root.minsize(450, 550)

        # --- CHARGEMENT DU MODÈLE IA ---
        try:
            self.ia_model = joblib.load('knn_model.joblib')
            self.label_encoder = joblib.load('label_encoder.joblib')
            self.ia_enabled = True
            print("Modèle IA chargé avec succès. ✅")
        except FileNotFoundError:
            self.ia_model = None
            self.label_encoder = None
            self.ia_enabled = False
            print("❌ Fichiers du modèle non trouvés. Le mode IA est désactivé.")

        # --- CONFIGURATION DE LA FENÊTRE AU CENTRE ---
        window_width = 600
        window_height = 700
        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()
        center_x = int(screen_width / 2 - window_width / 2)
        center_y = int(screen_height / 2 - window_height / 2)
        self.root.geometry(f'{window_width}x{window_height}+{center_x}+{center_y}')

        # --- PALETTE DE COULEURS & POLICES AMÉLIORÉES ---
        self.colors = {
            "bg": "#1a1a1a",
            "grid": "#00e5ff",      # Cyan vibrant
            "X": "#00e5ff",
            "O": "#ff40ff",      # Magenta vif
            "win_bg": "#2a3b4b", # Bleu foncé pour la victoire
            "text": "#ffffff",
            "btn_bg": "#222222",
            "btn_fg": "#00e5ff",
            "btn_hover_bg": "#00e5ff",
            "btn_hover_fg": "#000000",
            "btn_disabled_bg": "#1f1f1f",
            "btn_disabled_fg": "#555555"
        }
        
        # --- POLICES ---
        try:
            self.title_font = font.Font(family='Orbitron', size=36, weight='bold')
            self.main_font = font.Font(family='Orbitron', size=18, weight='bold')
            self.button_font = font.Font(family='Orbitron', size=14, weight='bold')
        except tk.TclError:
            print("Police 'Orbitron' non trouvée. Utilisation de polices par défaut.")
            self.title_font = font.Font(family='Arial', size=36, weight='bold')
            self.main_font = font.Font(family='Arial', size=18, weight='bold')
            self.button_font = font.Font(family='Arial', size=14, weight='bold')

        # --- INITIALISATION DE L'ÉTAT DU JEU ---
        self.board = [""] * 9
        self.cells = []
        self.player = "X"
        self.game_over = False
        self.game_mode = None
        self.resize_timer = None

        # --- CRÉATION DES ÉCRANS ---
        self.create_frames()
        self.show_main_menu()

    def create_frames(self):
        """Crée les conteneurs pour le menu et l'écran de jeu."""
        self.menu_frame = tk.Frame(self.root, bg="#0d0d0d")
        self.game_frame = tk.Frame(self.root, bg="#0d0d0d")
        self.create_main_menu_widgets()
        self.create_game_widgets()

    def create_main_menu_widgets(self):
        """Crée les widgets pour le menu principal."""
        title_frame = tk.Frame(self.menu_frame, bg="#0d0d0d")
        title_frame.pack(pady=(80, 60))
        tk.Label(title_frame, text="MORPION", font=self.title_font, bg="#0d0d0d", fg=self.colors["X"]).pack()
        tk.Label(title_frame, text="N É O N", font=self.title_font, bg="#0d0d0d", fg=self.colors["O"]).pack()
        
        buttons_container = tk.Frame(self.menu_frame, bg="#0d0d0d")
        buttons_container.pack(pady=20, padx=100, fill='x')

        pvp_button = self.create_styled_button(buttons_container, "Joueur vs Joueur", lambda: self.start_game('pvp'))
        pvp_button.pack(pady=10, ipady=10, fill='x')
        
        ia_button_state = 'normal' if self.ia_enabled else 'disabled'
        pve_button = self.create_styled_button(buttons_container, "Joueur vs IA", lambda: self.start_game('pve'), state=ia_button_state)
        pve_button.pack(pady=10, ipady=10, fill='x')
        
        if not self.ia_enabled:
            tk.Label(self.menu_frame, text="(Fichiers du modèle IA non trouvés)", font=(self.button_font.cget("family"), 10), bg="#0d0d0d", fg=self.colors["btn_disabled_fg"]).pack()
        else:
            tk.Label(self.menu_frame, text="(L'IA joue en tant que 'O')", font=(self.button_font.cget("family"), 10), bg="#0d0d0d", fg=self.colors["btn_disabled_fg"]).pack()

    def create_game_widgets(self):
        """Crée les widgets pour l'écran de jeu."""
        self.status_label = tk.Label(self.game_frame, text="", font=self.main_font, bg="#0d0d0d")
        self.status_label.pack(side="top", pady=20)

        self.board_container = tk.Frame(self.game_frame, bg="#0d0d0d")
        self.board_container.pack(fill="both", expand=True, padx=20, pady=20)
        self.board_container.bind("<Configure>", self.on_resize)
        
        self.board_frame = tk.Frame(self.board_container, bg=self.colors["grid"])
        
        for i in range(9):
            row, col = i // 3, i % 3
            cell_frame = tk.Frame(self.board_frame, bg=self.colors["bg"])
            cell_frame.grid(row=row, column=col, padx=3, pady=3, sticky="nsew")
            
            canvas = tk.Canvas(cell_frame, bg=self.colors["bg"], highlightthickness=0, relief='flat')
            canvas.pack(fill="both", expand=True)
            canvas.bind("<Button-1>", lambda event, index=i: self.on_cell_click(index))
            self.cells.append(canvas)

        for i in range(3):
            self.board_frame.grid_rowconfigure(i, weight=1)
            self.board_frame.grid_columnconfigure(i, weight=1)

        buttons_frame = tk.Frame(self.game_frame, bg="#0d0d0d")
        buttons_frame.pack(side="bottom", fill="x", pady=20, padx=20)
        buttons_frame.columnconfigure((0, 1), weight=1)

        self.restart_button = self.create_styled_button(buttons_frame, "Recommencer", self.restart_game)
        self.restart_button.grid(row=0, column=0, padx=5, ipady=5, sticky="ew")
        
        self.menu_button = self.create_styled_button(buttons_frame, "Menu Principal", self.show_main_menu)
        self.menu_button.grid(row=0, column=1, padx=5, ipady=5, sticky="ew")

    def on_resize(self, event):
        if self.resize_timer:
            self.root.after_cancel(self.resize_timer)
        self.resize_timer = self.root.after(50, self.perform_resize)

    def perform_resize(self, event=None):
        container_width = self.board_container.winfo_width()
        container_height = self.board_container.winfo_height()
        size = min(container_width, container_height)
        self.board_frame.place(in_=self.board_container, anchor="c", relx=.5, rely=.5, width=size, height=size)
        self.redraw_board()

    def redraw_board(self):
        for i, canvas in enumerate(self.cells):
            canvas.delete("all")
            if self.board[i]:
                self.draw_symbol(canvas, self.board[i])

    def create_styled_button(self, parent, text, command, state='normal'):
        bg_color = self.colors["btn_bg"] if state == 'normal' else self.colors["btn_disabled_bg"]
        fg_color = self.colors["btn_fg"] if state == 'normal' else self.colors["btn_disabled_fg"]

        button = tk.Button(
            parent, text=text, font=self.button_font,
            bg=bg_color, fg=fg_color,
            activebackground=self.colors["btn_hover_bg"], activeforeground=self.colors["btn_hover_fg"],
            command=command, relief="flat", borderwidth=2, state=state,
            disabledforeground=self.colors["btn_disabled_fg"]
        )

        if state == 'normal':
            button.bind("<Enter>", lambda e: e.widget.config(bg=self.colors["btn_hover_bg"], fg=self.colors["btn_hover_fg"]))
            button.bind("<Leave>", lambda e: e.widget.config(bg=self.colors["btn_bg"], fg=self.colors["btn_fg"]))
        
        return button

    def show_main_menu(self):
        self.game_frame.pack_forget()
        self.menu_frame.pack(fill="both", expand=True)

    def start_game(self, mode):
        self.game_mode = mode
        self.menu_frame.pack_forget()
        self.game_frame.pack(fill="both", expand=True)
        self.restart_game()
        self.root.after(10, self.perform_resize)

    ### CORRECTION 1 : Ajout d'un paramètre `is_ai_move` ###
    def on_cell_click(self, index, is_ai_move=False):
        """Gère le clic sur une case du plateau."""
        
        ### CORRECTION 2 : La condition de blocage ignore maintenant le coup de l'IA ###
        # Empêche le joueur de cliquer pendant le tour de l'IA, mais autorise le coup de l'IA.
        if self.game_mode == 'pve' and self.player == 'O' and not is_ai_move:
            return

        if self.board[index] == "" and not self.game_over:
            self.board[index] = self.player
            self.draw_symbol(self.cells[index], self.player)
            
            if self.check_winner():
                self.game_over = True
                self.update_status(winner=self.player)
            elif "" not in self.board:
                self.game_over = True
                self.update_status(draw=True)
            else:
                self.switch_player()
                self.update_status()
                # Si c'est le mode JvIA, que c'est au tour de 'O' et que la partie n'est pas finie,
                # on déclenche le tour de l'IA.
                if self.game_mode == 'pve' and self.player == 'O' and not self.game_over:
                    self.trigger_ai_move()

    def draw_symbol(self, canvas, player):
        canvas.update_idletasks() 
        w, h = canvas.winfo_width(), canvas.winfo_height()
        padding = w * 0.2
        color = self.colors[player]
        width = max(4, int(w * 0.1))

        if w < 20 or h < 20: return

        if player == "X":
            canvas.create_line(padding, padding, w - padding, h - padding, fill=color, width=width, capstyle=tk.ROUND)
            canvas.create_line(w - padding, padding, padding, h - padding, fill=color, width=width, capstyle=tk.ROUND)
        else:
            canvas.create_oval(padding, padding, w - padding, h - padding, outline=color, width=width)

    def check_winner(self):
        win_conditions = [[0, 1, 2], [3, 4, 5], [6, 7, 8], [0, 3, 6], [1, 4, 7], [2, 5, 8], [0, 4, 8], [2, 4, 6]]
        for condition in win_conditions:
            a, b, c = condition
            if self.board[a] == self.board[b] == self.board[c] != "":
                self.highlight_winner(condition)
                return True
        return False

    def highlight_winner(self, condition):
        for index in condition:
            self.cells[index].configure(bg=self.colors["win_bg"])
            self.cells[index].master.configure(bg=self.colors["win_bg"])

    def switch_player(self):
        self.player = "O" if self.player == "X" else "X"

    def update_status(self, winner=None, draw=False):
        if winner:
            self.status_label.config(text=f"Le Joueur {winner} a gagné !", fg=self.colors[winner])
        elif draw:
            self.status_label.config(text="Égalité !", fg=self.colors["text"])
        else:
            self.status_label.config(text=f"Au tour du Joueur {self.player}", fg=self.colors[self.player])

    def restart_game(self):
        self.board = [""] * 9
        self.player = "X"
        self.game_over = False
        for i, canvas in enumerate(self.cells):
            canvas.delete("all")
            canvas.configure(bg=self.colors["bg"])
            canvas.master.configure(bg=self.colors["bg"])
        self.update_status()
        self.redraw_board()

    def trigger_ai_move(self):
        """Prépare les données, demande à l'IA de jouer et exécute le coup."""
        self.root.config(cursor="watch")
        self.game_frame.update_idletasks()

        board_vector = np.array([self.board_to_vector(self.board)])
        predicted_move_encoded = self.ia_model.predict(board_vector)
        predicted_move_str = self.label_encoder.inverse_transform(predicted_move_encoded)[0]
        row, col = map(int, predicted_move_str.split(','))
        index = row * 3 + col

        if self.board[index] != "":
            empty_cells = [i for i, v in enumerate(self.board) if v == ""]
            if empty_cells:
                index = random.choice(empty_cells)

        ### CORRECTION 3 : On passe `is_ai_move=True` pour que le coup ne soit pas bloqué ###
        self.root.after(500, lambda: self.on_cell_click(index, is_ai_move=True))
        self.root.config(cursor="")

if __name__ == "__main__":
    root = tk.Tk()
    app = FuturisticTicTacToe(root)
    root.mainloop()


Modèle IA chargé avec succès. ✅
